In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import datetime
# from learntools.time_series import style

from pathlib import Path
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go

In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
stores = pd.read_csv("stores.csv")
oil = pd.read_csv("oil.csv")
transactions = pd.read_csv("transactions.csv")
holidays_events = pd.read_csv("holidays_events.csv")

In [4]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [5]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [6]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [7]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [8]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [9]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [10]:
pd.set_option('display.max_columns', None)
df_data = pd.concat([train, test], sort=True)
df_data = df_data.merge(stores, how="left", on='store_nbr')   
df_data = df_data.merge(oil, how="left", on='date')      
df_data = df_data.merge(transactions, how="left", on=['date','store_nbr'])  
df_data = df_data.merge(holidays_events,on='date',how='left')
df_data.head()

,date,family,id,onpromotion,sales,store_nbr,city,state,type_x,cluster,dcoilwtico,transactions,type_y,locale,locale_name,description,transferred
0,2013-01-01,AUTOMOTIVE,0,0,0.0,1,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-01,BABY CARE,1,0,0.0,1,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,2013-01-01,BEAUTY,2,0,0.0,1,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,2013-01-01,BEVERAGES,3,0,0.0,1,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,2013-01-01,BOOKS,4,0,0.0,1,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False


In [11]:
pd.set_option('display.max_columns', None)
df_data = pd.concat([train, test], sort=True)
df_data = df_data.merge(stores, how="left", on='store_nbr')   
df_data = df_data.merge(oil, how="left", on='date')      
df_data = df_data.merge(transactions, how="left", on=['date','store_nbr'])  
df_data = df_data.merge(holidays_events,on='date',how='left')
df_data = df_data.rename(columns={'type_x' : 'store_type','type_y':'holiday_type'})

In [12]:
pd.set_option('display.max_columns', None)
df_data = pd.concat([train, test], sort=True)
df_data = df_data.merge(stores, how="left", on='store_nbr')   
df_data = df_data.merge(oil, how="left", on='date')      
df_data = df_data.merge(transactions, how="left", on=['date','store_nbr'])  
df_data = df_data.merge(holidays_events,on='date',how='left')
df_data = df_data.rename(columns={'type_x' : 'store_type','type_y':'holiday_type'})

df_data.date = pd.to_datetime(df_data.date)
df_data['year'] = df_data['date'].dt.year
df_data['month'] = df_data['date'].dt.month
df_data['week'] = df_data['date'].dt.isocalendar().week
df_data['quarter'] = df_data['date'].dt.quarter
df_data['day_of_week'] = df_data['date'].dt.day_name()
df_data.head()

,date,family,id,onpromotion,sales,store_nbr,city,state,store_type,cluster,dcoilwtico,transactions,holiday_type,locale,locale_name,description,transferred,year,month,week,quarter,day_of_week
0,2013-01-01,AUTOMOTIVE,0,0,0.0,1,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False,2013,1,1,1,Tuesday
1,2013-01-01,BABY CARE,1,0,0.0,1,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False,2013,1,1,1,Tuesday
2,2013-01-01,BEAUTY,2,0,0.0,1,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False,2013,1,1,1,Tuesday
3,2013-01-01,BEVERAGES,3,0,0.0,1,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False,2013,1,1,1,Tuesday
4,2013-01-01,BOOKS,4,0,0.0,1,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False,2013,1,1,1,Tuesday


In [13]:
df_data.isnull().sum()

date                  0
family                0
id                    0
onpromotion           0
sales             28512
store_nbr             0
city                  0
state                 0
store_type            0
cluster               0
dcoilwtico       962280
transactions     277629
holiday_type    2578554
locale          2578554
locale_name     2578554
description     2578554
transferred     2578554
year                  0
month                 0
week                  0
quarter               0
day_of_week           0
dtype: int64

In [14]:
df_data[df_data["sales"].isnull()]

,date,family,id,onpromotion,sales,store_nbr,city,state,store_type,cluster,dcoilwtico,transactions,holiday_type,locale,locale_name,description,transferred,year,month,week,quarter,day_of_week
3054348,2017-08-16,AUTOMOTIVE,3000888,0,NaN,1,Quito,Pichincha,D,13,46.80,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,33,3,Wednesday
3054349,2017-08-16,BABY CARE,3000889,0,NaN,1,Quito,Pichincha,D,13,46.80,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,33,3,Wednesday
3054350,2017-08-16,BEAUTY,3000890,2,NaN,1,Quito,Pichincha,D,13,46.80,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,33,3,Wednesday
3054351,2017-08-16,BEVERAGES,3000891,20,NaN,1,Quito,Pichincha,D,13,46.80,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,33,3,Wednesday
3054352,2017-08-16,BOOKS,3000892,0,NaN,1,Quito,Pichincha,D,13,46.80,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,33,3,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082855,2017-08-31,POULTRY,3029395,1,NaN,9,Quito,Pichincha,B,6,47.26,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,35,3,Thursday
3082856,2017-08-31,PREPARED FOODS,3029396,0,NaN,9,Quito,Pichincha,B,6,47.26,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,35,3,Thursday
3082857,2017-08-31,PRODUCE,3029397,1,NaN,9,Quito,Pichincha,B,6,47.26,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,35,3,Thursday
3082858,2017-08-31,SCHOOL AND OFFICE SUPPLIES,3029398,9,NaN,9,Quito,Pichincha,B,6,47.26,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,35,3,Thursday


In [15]:
combinations_count = df_data[['store_nbr', 'family', "onpromotion"]].drop_duplicates().shape[0]
combinations_count

36657

In [16]:
df_data.isnull().sum()

date                  0
family                0
id                    0
onpromotion           0
sales             28512
store_nbr             0
city                  0
state                 0
store_type            0
cluster               0
dcoilwtico       962280
transactions     277629
holiday_type    2578554
locale          2578554
locale_name     2578554
description     2578554
transferred     2578554
year                  0
month                 0
week                  0
quarter               0
day_of_week           0
dtype: int64

## Dealing with Null Value
### Oil Price

In [17]:
# Fill missing value by backward fill technique (filling missing data with next data point)
df_data["dcoilwtico"] = df_data["dcoilwtico"].fillna(method="bfill")
df_data.dcoilwtico.isnull().sum()

C:\Users\Jessica\AppData\Local\Temp\ipykernel_23072\4200159608.py:2: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



0

In [18]:
df_data

,date,family,id,onpromotion,sales,store_nbr,city,state,store_type,cluster,dcoilwtico,transactions,holiday_type,locale,locale_name,description,transferred,year,month,week,quarter,day_of_week
0,2013-01-01,AUTOMOTIVE,0,0,0.0,1,Quito,Pichincha,D,13,93.14,NaN,Holiday,National,Ecuador,Primer dia del ano,False,2013,1,1,1,Tuesday
1,2013-01-01,BABY CARE,1,0,0.0,1,Quito,Pichincha,D,13,93.14,NaN,Holiday,National,Ecuador,Primer dia del ano,False,2013,1,1,1,Tuesday
2,2013-01-01,BEAUTY,2,0,0.0,1,Quito,Pichincha,D,13,93.14,NaN,Holiday,National,Ecuador,Primer dia del ano,False,2013,1,1,1,Tuesday
3,2013-01-01,BEVERAGES,3,0,0.0,1,Quito,Pichincha,D,13,93.14,NaN,Holiday,National,Ecuador,Primer dia del ano,False,2013,1,1,1,Tuesday
4,2013-01-01,BOOKS,4,0,0.0,1,Quito,Pichincha,D,13,93.14,NaN,Holiday,National,Ecuador,Primer dia del ano,False,2013,1,1,1,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082855,2017-08-31,POULTRY,3029395,1,NaN,9,Quito,Pichincha,B,6,47.26,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,35,3,Thursday
3082856,2017-08-31,PREPARED FOODS,3029396,0,NaN,9,Quito,Pichincha,B,6,47.26,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,35,3,Thursday
3082857,2017-08-31,PRODUCE,3029397,1,NaN,9,Quito,Pichincha,B,6,47.26,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,35,3,Thursday
3082858,2017-08-31,SCHOOL AND OFFICE SUPPLIES,3029398,9,NaN,9,Quito,Pichincha,B,6,47.26,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,35,3,Thursday


### Transaction

In [19]:
df_data.transactions = df_data.transactions.replace(np.nan,0)

### Holidays

In [20]:
df_data[["locale", "locale_name","description"]] = df_data[["locale", 
        "locale_name", "description"]].replace(np.nan, "")
df_data["holiday_type"] = df_data["holiday_type"].replace(np.nan, "Working Day")
df_data["transferred"] = df_data["transferred"].replace(np.nan, False)

### Output the dataframe

In [21]:
df_data.to_csv("store_item.csv", index = False)